In [ ]:
#레퍼런스
# 강의    :  https://www.youtube.com/watch?v=zAVCeF5cFNc
# github  :  https://github.com/hanyoseob

In [ ]:
class CBR2d(nn.Module):
    def __init__(self,in_channels,out_channels, kernel_size=3, stride=1, padding=1, bias=True,norm="bnorm",relu =0.0):
      super().__init__()
      
      layers = []
      layers+=[nn.ReflectionPad2d(padding=padding)]
      layers+=[nn.Conv2d(in_channels =in_channels,out_channels=out_channels,
                         kernel_size=kernel_size,stride=stride,padding=0,
                         bias=bias)]
     # layers+=[nn.BatchNorm2d(num_features=out_channels)]
     # layers+=[nn.Relu()]

      if not norm is None:
        if norm == "bnorm":
          layers +=[nn.BatchNorm2d(num_features=out_channels)]
        elif norm == "inorm":
          layers+=[nn.InstancedNorm2d(num_features=out_channels)]

      if not relu is None and relu >= 0.0:
        layers+=[nn.ReLU() if relu == 0 else nn.LeakyRelu(relu)]


      self.cbr =nn.Sequential(*layers)

    def forward(self,x):
      return self.cbr(x)



class DECBR2d(nn.Module):
    def __init__(self,in_channels,out_channels, kernel_size=3, stride=1, padding=1, bias=True,norm="bnorm",relu =0.0):
      super().__init__()
      
      layers = []
      layers+=[nn.ReflectionPad2d(padding=padding)]
      layers+=[nn.Conv2d(in_channels =in_channels,out_channels=out_channels,
                         kernel_size=kernel_size,stride=stride,padding=padding,
                         bias=bias)]
     # layers+=[nn.BatchNorm2d(num_features=out_channels)]
     # layers+=[nn.Relu()]

      if not norm is None:
        if norm == "bnorm":
          layers +=[nn.BatchNorm2d(num_features=out_channels)]
        elif norm == "inorm":
          layers+=[nn.InstancedNorm2d(num_features=out_channels)]

      if not relu is None and relu >= 0.0:
        layers+=[nn.ReLU() if relu == 0 else nn.LeakyRelu(relu)]


      self.cbr =nn.Sequential(*layers)

    def forward(self,x):
      return self.cbr(x)


class ResBlock(nn.Module):
    def __init__(self,in_channels,out_channels, kernel_size=3, stride=1, padding=1, bias=True, norm="bnorm",relu=0.0):
        super().__init__()

        layers = []

        # 1st conv
      
        layers += [CBR2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, norm=norm, relu=relu)]


        # 2nd conv
        layers += [Padding(padding, padding_mode=padding_mode)]
        layers += [CNR2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding,bias=bias, norm=norm, relu=None)]

        self.resblk = nn.Sequential(*layers)

    def forward(self, x):
        return x + self.resblk(x)








In [ ]:
class CycleGAN(nn.Module):
  def__init__(self,in_channels,out_channels,nker=64,norm='inorm'):
    super(CycleGAN,self).__init()

    self.enc1 = CBR2d(in_channels,1*nker,kernel_size=7,padding=(7-1)/2,norm=norm,relu=0.0,stride=1)
    self.enc2 = CBR2d(1*nker,2*nker,kernel_size=3,padding=(3-1)/2,norm=norm,relu=0.0,stride=2)
    self.enc3 = CBR2d(2*nker,4*nker,kernel_size=3,padding=(3-1)/2,norm=norm,relu=0.0,stride=2)

    res = []
    for i in range(9):
      res += [ResBlock(4*nker,4*nker,kernel_size=3, padding=(3-1)/2),norm=norm,relu=0.0,stride=1)]
    self.res = nn.Sequential(*res)

    self.dec1=DECBR2d(4*nker,2*nker,kernel_size=3,padding=(3-1)/2,norm=norm,relu=0.0,stride=2)
    self.dec2=DECBR2d(2*nker,1*nker,kernel_size=3,padding=(3-1)/2,norm=norm,relu=0.0,stride=2)
    self.dec3 =CBR2d(1*nker,out_channels,kernel_size=7,padding=(7-1)/2,norm=None,relu=None,stride=1)



  def forward(self,x):
    x=self.enc1(x)
    x=self.enc2(x)
    x=self.enc3(x)

    x=self.res(x)

    x=self.dec1(x)
    x=self.dec2(x)
    x=self.dec3(x)

    x= torch.tanh(x)

    return x




class Discriminator(nn.Module):
    def __init__(self, in_channels,out_channels,nker=64, norm='bnorm'):
        super(Discriminator, self).__init__()

        self.enc1 = CBR2d(1 * in_channels,  1 * nker, kernel_size=4, stride=2, padding=1, norm=None, relu=0.2,bias=False)
        self.enc2 = CBR2d(1 * nker, 2 * nker, kernel_size=4, stride=2, padding=1, norm=norm, relu=0.2, bias=False)
        self.enc3 = CBR2d(2 * nker, 4 * nker, kernel_size=4, stride=2, padding=1, norm=norm, relu=0.2, bias=False)
        self.enc4 = CBR2d(4 * nker, 8 * nker, kernel_size=4, stride=2, padding=1, norm=norm, relu=0.2, bias=False)
        self.enc5 = CBR2d(8 * nker, out_channels, kernel_size=4, stride=2, padding=1, norm=None, relu=None, bias=False)


    def forward(self, x):

        x = self.enc1(x)
        x = self.enc2(x)
        x = self.enc3(x)
        x = self.enc4(x)
        x = self.enc5(x)

        x = torch.sigmoid(x)

        return x

